In [1]:
import pandas as pd
from datetime import datetime
import os

In [2]:
# Define the influence factors for each time period
influence_periods = [
    # May 26th, 2022 - January 25th, 2023
    {
        "start_date": "2022-05-26", "end_date": "2023-01-25",
        "influences": {"th_vp": 48.32, "ch_vp_r2": 51.68}
    },
    # January 26th, 2023 - March 30th, 2023
    {
        "start_date": "2023-01-26", "end_date": "2023-03-30",
        "influences": {"th_vp": 41.95, "ch_vp_r2": 44.88, "gc_vp_s3": 13.17}
    },
    # March 31st, 2023 - June 7th, 2023
    {
        "start_date": "2023-03-31", "end_date": "2023-06-07",
        "influences": {"th_vp": 41.95, "ch_vp_r3": 44.88, "gc_vp_s3": 13.17}
    },
    # June 8th, 2023 - January 3rd, 2024
    {
        "start_date": "2023-06-08", "end_date": "2024-01-03",
        "influences": {"th_vp": 41.95, "ch_vp_r3": 44.88, "gc_vp_s4": 13.17}
    },
    # January 4th, 2024 - January 11th, 2024
    {
        "start_date": "2024-01-04", "end_date": "2024-01-11",
        "influences": {
            "th_vp": 32.33, "ch_vp_r3": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32,
            "dab_vp_s5": 3.01
        }
    },
    # January 12th, 2024 - June 26th, 2024
    {
        "start_date": "2024-01-12", "end_date": "2024-06-26",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s5": 10.15,
            "gc_vp_mm_s5": 2.82, "sc_vp_s5": 12.78, "coc_vp_s5": 4.32,
            "dab_vp_s5": 3.01
        }
    },
    # June 27th, 2024 - July 16th, 2024
    {
        "start_date": "2024-06-27", "end_date": "2024-07-16",
        "influences": {
            "th_vp": 32.33, "ch_vp_r4": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # July 17th, 2024 - October 21st, 2024
    {
        "start_date": "2024-07-17", "end_date": "2024-10-21",
        "influences": {
            "th_vp": 32.33, "ch_vp_r5": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # October 22nd, 2024 - December 12th, 2024
    {
        "start_date": "2024-10-22", "end_date": "2024-12-12",
        "influences": {
            "th_vp": 32.33, "ch_vp_r6": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # December 13th, 2024 - January 15th, 2025
    {
        "start_date": "2024-12-13", "end_date": "2025-01-15",
        "influences": {
            "th_vp": 32.33, "ch_vp_r7": 34.59, "gc_vp_s6": 10.15,
            "gc_vp_mm_s6": 2.82, "sc_vp_s6": 12.78, "coc_vp_s6": 4.32,
            "dab_vp_s6": 3.01
        }
    },
    # January 16th, 2025 - July 23rd, 2025
    {
        "start_date": "2025-01-16", "end_date": "2025-07-23",
        "influences": {
            "th_vp": 33.73, "ch_vp_r7": 36.08, "gc_vp_s7": 10.59,
            "gc_vp_op_s7": 0.19, "sc_vp_s7": 13.33, "dab_vp_s7": 3.14,
            "mmc_vp_s7": 2.94
        }
    }
]

In [3]:
def calculate_influence(row, influences):
    """Calculates the influence for a row based on the given influence percentages."""
    influence_sum = 0
    for column, influence_value in influences.items():
        influence_sum += row.get(column, 0) * (influence_value / 100)
    return influence_sum

In [4]:
def add_influence_column(df, file_date_str):
    """Adds the 'influence' column to the DataFrame based on the file date."""
    # Convert file_date_str to datetime object
    file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
    
    # Find the correct influence period based on the file date
    for period in influence_periods:
        start_date = datetime.strptime(period["start_date"], "%Y-%m-%d")
        end_date = datetime.strptime(period["end_date"], "%Y-%m-%d")
        if start_date <= file_date <= end_date:
            # Calculate the influence column for each row
            df["influence"] = df.apply(calculate_influence, axis=1, influences=period["influences"])
            break
    
    return df

In [5]:
def process_file(file_path, file_date_str):
    """Process a single CSV file: add influence column, calculate HHI and mHHI."""
    data = pd.read_csv(file_path)

    # Add the influence column
    data = add_influence_column(data, file_date_str)

    # Calculate squared values for HHI and CPI
    data['th_vp_squared'] = data['th_vp'] ** 2
    data['influence_squared'] = data['influence'] ** 2

    # Calculate HHI and CPI
    HHI = round(data['th_vp_squared'].sum(), 2)
    CPI = round(data['influence_squared'].sum(), 2)

    # Log progress
    print(f"File: {file_path} | Date: {file_date_str} | HHI: {HHI} | CPI: {CPI}")
    
    return file_date_str, HHI, CPI

In [6]:
def append_results_to_csv(output_path, new_results):
    """Appends the new results to the CSV file, creating the file if it doesn't exist."""
    if os.path.exists(output_path):
        # Load the existing results
        existing_df = pd.read_csv(output_path)
    else:
        # Create an empty DataFrame with the necessary columns
        existing_df = pd.DataFrame(columns=["date", "HHI", "CPI"])

    # Convert the 'date' column to datetime to compare dates correctly
    existing_df['date'] = pd.to_datetime(existing_df['date'], format='%Y-%m-%d')

    # Convert new_results to a DataFrame
    new_results_df = pd.DataFrame(new_results, columns=["date", "HHI", "CPI"])
    new_results_df['date'] = pd.to_datetime(new_results_df['date'], format='%Y-%m-%d')

    # Find dates that are not already in the existing DataFrame
    new_results_df = new_results_df[~new_results_df['date'].isin(existing_df['date'])]

    # Append the new results to the existing DataFrame
    combined_df = pd.concat([existing_df, new_results_df], ignore_index=True)

    # Save the updated DataFrame back to the CSV file
    combined_df.to_csv(output_path, index=False)

In [7]:
def process_files_in_folder(folder_path, output_path):
    """Process all CSV files in the specified folder and append HHI/mHHI results to the CSV."""
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    new_results = []

    for file_name in csv_files:
        file_path = os.path.join(folder_path, file_name)
        file_date_str = file_name[:-4]  # Extract date from file name

        # Process the file and get HHI and CPI
        date, HHI, CPI = process_file(file_path, file_date_str)
        new_results.append([date, HHI, CPI])

    # Append the new results to the output CSV file
    append_results_to_csv(output_path, new_results)
    print(f"New results appended to {output_path}")

In [8]:
# Example usage:
process_files_in_folder("../Data/Data_Sheets/", "../Data/daily_hhi_and_cpi_new.csv")

File: ../Data/Data_Sheets/2022-05-26.csv | Date: 2022-05-26 | HHI: 0 | CPI: 36.59
File: ../Data/Data_Sheets/2022-05-27.csv | Date: 2022-05-27 | HHI: 10000.0 | CPI: 2371.41
File: ../Data/Data_Sheets/2022-05-28.csv | Date: 2022-05-28 | HHI: 10000.0 | CPI: 2371.41
File: ../Data/Data_Sheets/2022-05-29.csv | Date: 2022-05-29 | HHI: 10000.0 | CPI: 2371.41
File: ../Data/Data_Sheets/2022-05-30.csv | Date: 2022-05-30 | HHI: 10000.0 | CPI: 2371.41
File: ../Data/Data_Sheets/2022-05-31.csv | Date: 2022-05-31 | HHI: 624.17 | CPI: 211.55
File: ../Data/Data_Sheets/2022-06-01.csv | Date: 2022-06-01 | HHI: 352.25 | CPI: 143.7
File: ../Data/Data_Sheets/2022-06-02.csv | Date: 2022-06-02 | HHI: 365.84 | CPI: 147.23
File: ../Data/Data_Sheets/2022-06-03.csv | Date: 2022-06-03 | HHI: 357.33 | CPI: 144.18
File: ../Data/Data_Sheets/2022-06-04.csv | Date: 2022-06-04 | HHI: 365.04 | CPI: 146.21
File: ../Data/Data_Sheets/2022-06-05.csv | Date: 2022-06-05 | HHI: 364.77 | CPI: 146.38
File: ../Data/Data_Sheets/2022-

C:\Users\Dhruvi\AppData\Local\Temp\ipykernel_7360\697249526.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, new_results_df], ignore_index=True)
